In [1]:
import script.compile as compiler
from script.model import model_cal, grid_search

In [2]:
data, desc, cols = compiler.data_loader(convert = False)

In [3]:
model_cal(data, cols, shap = False)

conv


KeyboardInterrupt: 

In [6]:
feat, target = data.loc[:, cols['prop2']], data.loc[:, cols['target']]
feat

,IWI,WI,CI,SI,SGP,CP,HDP,UGC,SCT,FSP,...,AN (4),AW (4),atomic radius (4),electronegativity (4),m. p. (4),b. p. (4),delta_fus H (4),density (4),ionization enegy (4),Surface energy (4)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4185,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4186,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4187,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
def smac(model, init_x, init_y, roen_func, desc, cols, model_type, random_state = 1126):
    """
    SMAC　のメインプログラム
    """
    model.fit(init_x, init_y)
    print(model)
    mu, sigma =  posterior(init_x, init_x, init_y, model)
    ei =  EI(mu, sigma, init_y.max())
    ei = pd.Series(ei, index = init_x.index, name = 'ei')
    make_nei =pd.Series(True, index=init_x.index, name='make_nei')
    next_x = pd.concat([init_x, ei, make_nei], axis =1)
    while next_x['make_nei'].sum() != 0:
        next_x = roen_func(next_x, init_y, model, desc)
        print(next_x['make_nei'].sum())
        
    return next_x

def opt_function(x, y ,model, desc,random_state =1126):
    """
    各実験ずみの点に対して4回近傍の探索を行い最もEIの高かった組成実験条件を返す
    """
    xx = x.iloc[:,:-2]
    neighbor = pd.DataFrame(index = x.columns)
    for key, row in xx.iterrows():
        count = 0
        nei_cand = []
        #print('ind={0}'.format(key))
        if x.loc[key, 'make_nei'] == True:

            for _ in range(30):
                row_ch = row.copy()
                
                row_ch = SWED_change(row_ch, desc)

                nei_cand.append(row_ch)
                
            for _ in range(30):
                row_ch = row.copy()
                
                row_ch = experiment_change(row_ch)
                
                nei_cand.append(row_ch)
                
            for _ in range(30):
                row_ch = row.copy()
                
                row_ch = SWED_change(row_ch, desc)
                
                row_ch = experiment_change(row_ch)
                
                nei_cand.append(row_ch)
                   

            nei_cand = pd.DataFrame(nei_cand, index =np.arange(len(nei_cand))).fillna(0)
            swed = comp_times_base(nei_cand.loc[:,element],desc.loc[element].T,sort=True,times=True)
            swed = pd.DataFrame(swed)
            swed = swed.iloc[:, :12].fillna(0)
            swed.columns = name2
            nei_cand.loc[:, name2] =swed
                        
            mu, sigma = model.predict(np.array(nei_cand.loc[:, with_swed2]), return_std=True)
            ind = y.values.argmax()
            cur_max = y.iloc[ind]
            ei = EI(mu, sigma, cur_max)
            ind = np.argmax(ei)
            cand = nei_cand.iloc[ind].copy()
            cand['ei'] = ei[ind]
            if x.loc[key, 'ei'] < cand['ei']:
                cand['make_nei'] = True
                neighbor = pd.concat([neighbor, cand], axis = 1)
            else:
                x.loc[key, 'make_nei'] = False
                neighbor = pd.concat([neighbor, x.loc[key,:]], axis = 1)
                
        else:
            neighbor = pd.concat([neighbor, x.loc[key,:]], axis = 1)
                                                    
    print('-----------')
    
    neighbor = neighbor.T
    neighbor.index = x.index
    return neighbor



In [7]:
cols.keys()

dict_keys(['element', 'preparation', 'condition', 'use_cols', 'swed', 'conv', 'prop1', 'prop2', 'target'])